In [1]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle

import pandas as pd
import openpyxl
import xgboost as xgb
from sklearn import metrics
import sklearn
import shap
import seaborn as sns
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [3]:
data_num = 470
feature_num = 16

features = np.zeros((data_num,feature_num))
label = np.zeros((data_num,))

In [4]:
data_location = os.getcwd()+"/../../1.original/real_5_TS/data.xlsx"
raw_data = openpyxl.load_workbook(data_location)
raw_data = raw_data.active

In [9]:
for data_index in range(data_num):
    
    # feature 1
    temp = int(raw_data.cell(row=data_index+22, column=1).value[3])
    if temp == 8:
        temp = 7
    features[data_index,0] = temp
    
    # feature 2, feature 3
    features[data_index,1] = raw_data.cell(row=data_index+22, column=2).value
    features[data_index,2] = raw_data.cell(row=data_index+22, column=3).value
    
    # feature 4
    features[data_index,3] = int(raw_data.cell(row=data_index+22, column=4).value[3])
    
    
    # feature 5-9
    for index_temp in range(5):
        if raw_data.cell(row=data_index+22, column=5+index_temp).value == "T":
            features[data_index,4+index_temp] = 1
        elif raw_data.cell(row=data_index+22, column=5+index_temp).value == "F":
            features[data_index,4+index_temp] = 0
        else:
            print("Error")
    # feature 10
    features[data_index,9] = int(raw_data.cell(row=data_index+22, column=10).value[3])
    
    # feature 11-15
    for index_temp in range(5):
        if raw_data.cell(row=data_index+22, column=11+index_temp).value == "T":
            features[data_index,10+index_temp] = 1
        elif raw_data.cell(row=data_index+22, column=11+index_temp).value == "F":
            features[data_index,10+index_temp] = 0
        else:
            print("Error")
    # feature 16
    features[data_index,15] = raw_data.cell(row=data_index+22, column=16).value
    
    if raw_data.cell(row=data_index+22, column=17).value == "T":
        label[data_index]  = 1
    elif raw_data.cell(row=data_index+22, column=17).value == "F":
        label[data_index] = 0
    else:
        print("Error")


In [10]:
print(features)

[[ 2.    2.88  2.16 ...  1.    0.   60.  ]
 [ 3.    3.4   1.88 ...  1.    0.   51.  ]
 [ 3.    2.76  2.08 ...  1.    0.   59.  ]
 ...
 [ 3.    3.04  2.08 ...  0.    0.   52.  ]
 [ 3.    1.96  1.68 ...  1.    0.   79.  ]
 [ 3.    4.72  3.56 ...  1.    0.   51.  ]]


In [11]:
print(label)

[0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.

In [12]:
scaler = MinMaxScaler()
scaler.fit(features)
features = scaler.transform(features)

In [13]:
np.save("features.npy",features)
np.save("label.npy",label)